In [2]:
import pandas as pd
import numpy as np
import pickle
import nltk
import re
import string
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import spacy
from collections import Counter
import scipy.sparse as ss
from sklearn import datasets
from corextopic import corextopic as ct
from corextopic import vis_topic as vt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
# Read in the transcripts
with open('transcripts.pickle','rb') as read_file:
    transcripts = pickle.load(read_file)

## Create df of 5mins of text/ row

In [73]:
def bin_5min_blocks(df, key):
    text_list = []
    time_list = []
    key_list = []
    i=0
    while i < len(df):
        current_time = df.start.iloc[i]
        current_text = ''
        while (i < len(df)) and (df.start.iloc[i] <= current_time+300):
            current_text = current_text + ' ' + df.text.iloc[i]
            i +=1
        time_list.append(current_time)
        text_list.append(current_text)
        key_list.append(key)
    return pd.DataFrame(list(zip(key_list, time_list, text_list)),\
                        columns=['episode', 'time','text'])

In [80]:
binned_df = pd.DataFrame(columns=['episode','time','text'])
for key in transcripts.keys():
    binned_df = binned_df.append(bin_5min_blocks(pd.DataFrame(transcripts[key]), key))

In [81]:
binned_df.shape

(2964, 3)

In [82]:
# Pickle the 5 min bin df
with open('binned_df.pickle', 'wb') as to_write:
    pickle.dump(binned_df, to_write)

# spaCy

In [14]:
# Read in the bin df
with open('binned_df.pickle','rb') as read_file:
    binned_df = pickle.load(read_file)

binned_df = binned_df.reset_index().drop(columns='index')
binned_df

/Users/Jay/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,episode,time,text
0,xXXIACCJ2io,60.399,welcome to the podcast that's dedicated to ma...
1,xXXIACCJ2io,360.479,i mean why not the floor is ours another thin...
2,xXXIACCJ2io,661.760,comprehensive understanding of my own injurie...
3,xXXIACCJ2io,962.399,that and you probably never did it again so t...
4,xXXIACCJ2io,1263.360,assumption of what's going on with my body bu...
...,...,...,...
2959,eWZVAazaDco,2711.030,like I need some something here and and so it...
2960,eWZVAazaDco,3012.300,of that is if you ever watched cross country ...
2961,eWZVAazaDco,3313.559,not like he was a tactical magician but peopl...
2962,eWZVAazaDco,3614.590,had a you know I had our day off within the l...


In [3]:
# Load spaCy's english core module

nlp = spacy.load('en_core_web_sm')

### Tokenize

In [6]:
# Tokenization (takes 5-10 min to run)

binned_df['spacy_doc'] = list(nlp.pipe(binned_df.text))

# to tokenize one string:
# doc = nlp(text)

In [7]:
# Pickle the 5 min bin tokenized text (1.59 GB)
with open('token_5min_df.pickle', 'wb') as to_write:
    pickle.dump(binned_df, to_write)

In [13]:
# Read in the 5 min tokenized text
with open('token_5min_df.pickle','rb') as read_file:
    binned_df = pickle.load(read_file)

/Users/Jay/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
binned_df

,episode,time,text,spacy_doc
0,xXXIACCJ2io,60.399,welcome to the podcast that's dedicated to ma...,"( , welcome, to, the, podcast, that, 's, dedic..."
1,xXXIACCJ2io,360.479,i mean why not the floor is ours another thin...,"( , i, mean, why, not, the, floor, is, ours, a..."
2,xXXIACCJ2io,661.760,comprehensive understanding of my own injurie...,"( , comprehensive, understanding, of, my, own,..."
3,xXXIACCJ2io,962.399,that and you probably never did it again so t...,"( , that, and, you, probably, never, did, it, ..."
4,xXXIACCJ2io,1263.360,assumption of what's going on with my body bu...,"( , assumption, of, what, 's, going, on, with,..."
...,...,...,...,...
2959,eWZVAazaDco,2711.030,like I need some something here and and so it...,"( , like, I, need, some, something, here, and,..."
2960,eWZVAazaDco,3012.300,of that is if you ever watched cross country ...,"( , of, that, is, if, you, ever, watched, cros..."
2961,eWZVAazaDco,3313.559,not like he was a tactical magician but peopl...,"( , not, like, he, was, a, tactical, magician,..."
2962,eWZVAazaDco,3614.590,had a you know I had our day off within the l...,"( , had, a, you, know, I, had, our, day, off, ..."


### Put Nouns and Adjectives in a string for each doc

In [40]:
def key_words(row):
    current_string = ''
    for token in row['spacy_doc']:
        if (token.is_stop == False) and ((token.pos_ == 'NOUN') or (token.pos_ == 'ADJ')):
            current_word = token.lemma_
            current_string = current_string + current_word + ' '
    return current_string

In [42]:
binned_df['key_words'] = binned_df.apply(key_words ,axis=1)

# Vectorize

In [44]:
vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b')
dtm = vectorizer.fit_transform(binned_df.key_words)

# don't actually need a df of the vectorized data...
#dtm_df = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names())

In [46]:
dtm_df.shape

(2964, 13981)

# Fit LDA Model

In [47]:
lda = LatentDirichletAllocation(n_components=20, random_state=0)
lda.fit(dtm)

LatentDirichletAllocation(n_components=20, random_state=0)

### Pickle Model, dtm and vectorizer

In [49]:
# Pickle the LDA Model, dtm, and vectorizer

mmv = [lda, dtm, vectorizer]

with open('mmv_5min.pickle', 'wb') as to_write:
    pickle.dump(mmv, to_write)

In [3]:
# Read in the LDA MOdel
with open('mmv_5min.pickle','rb') as read_file:
    mmv = pickle.load(read_file)

lda = mmv[0]
dtm = mmv[1]
vectorizer = mmv[2]

### pyLDAvis

In [4]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, dtm, vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.042319  0.133936       1        1  15.541911
14    -0.083372 -0.050525       2        1  15.125855
8     -0.041582  0.025570       3        1   9.482306
9     -0.107338  0.126712       4        1   9.036533
1     -0.113316  0.234590       5        1   7.065121
10     0.044584  0.200156       6        1   6.869066
3      0.060179 -0.069539       7        1   4.817982
11    -0.164517  0.146662       8        1   4.479757
2      0.071868  0.122143       9        1   4.130840
4      0.206966  0.165358      10        1   3.561332
6     -0.218957 -0.031905      11        1   3.517083
15     0.116800 -0.173830      12        1   3.455351
19    -0.330442  0.064384      13        1   2.956287
7      0.269640 -0.213531      14        1   2.863982
16     0.195606  0.006582      15        1   2.421532
5      0.309337  0.060941      16        1   1.989355
13     0.146876  0.315297      17        1   1.609269
17     0.079328 -0.319912      18        1   0.707478
12    -0.281221 -0.359350      19        1   0.189834
18    -0.118118 -0.383741      20        1   0.179125, topic_info=               Term         Freq        Total Category  logprob  loglift
13840       workout  4602.000000  4602.000000  Default  30.0000  30.0000
7687         minute  3554.000000  3554.000000  Default  29.0000  29.0000
9297          power  3912.000000  3912.000000  Default  28.0000  28.0000
1104           bike  6590.000000  6590.000000  Default  27.0000  27.0000
9757           race  9236.000000  9236.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
10079  relationship     2.753458   122.742619  Topic20  -5.9404   2.5276
5985          ideal     2.868166   195.643470  Topic20  -5.8996   2.1022
7687         minute     3.759155  3554.562976  Topic20  -5.6291  -0.5270
1104           bike     3.901863  6590.595266  Topic20  -5.5918  -1.1071
6340      intensity     3.254079  1217.975087  Topic20  -5.7733   0.3998

[1529 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
11         1  0.175381     able
11         2  0.146288     able
11         3  0.185216     able
11         4  0.081134     able
11         5  0.045484     able
...      ...       ...      ...
13944      4  0.035926  youtube
13944      6  0.080834  youtube
13944      7  0.035926  youtube
13944      8  0.008982  youtube
13944     17  0.017963  youtube

[5732 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 15, 9, 10, 2, 11, 4, 12, 3, 5, 7, 16, 20, 8, 17, 6, 14, 18, 13, 19])

In [19]:
# Get topic weights for each document
weights = lda.transform(dtm)

# make weight readable
topic_list = ['welcome','training','power_time','race','nutrition','terrain','medical','road_tactics','road_equip','technique',\
              'test_optim','heat','intervals','physiology','cadence','strength','mtb_equip','marginal_gains','protein','warmup']
weights_df = pd.DataFrame(weights, columns = topic_list)
doc_topic = pd.concat([binned_df[['episode','time']], weights_df], axis=1)


/Users/Jay/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
doc_topic

/Users/Jay/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,episode,time,welcome,training,power_time,race,nutrition,terrain,medical,road_tactics,...,test_optim,heat,intervals,physiology,cadence,strength,mtb_equip,marginal_gains,protein,warmup
0,xXXIACCJ2io,60.399,0.845051,0.000208,0.000208,0.000208,0.000208,0.000208,0.000208,0.000208,...,0.110485,0.040922,0.000208,0.000208,0.000208,0.000208,0.000208,0.000208,0.000208,0.000208
1,xXXIACCJ2io,360.479,0.519756,0.066363,0.000258,0.363240,0.000258,0.014569,0.000258,0.000258,...,0.000258,0.000258,0.000258,0.000258,0.032206,0.000258,0.000258,0.000258,0.000258,0.000258
2,xXXIACCJ2io,661.760,0.100954,0.000259,0.000259,0.894383,0.000259,0.000259,0.000259,0.000259,...,0.000259,0.000259,0.000259,0.000259,0.000259,0.000259,0.000259,0.000259,0.000259,0.000259
3,xXXIACCJ2io,962.399,0.000243,0.000243,0.000243,0.398645,0.207422,0.057193,0.000243,0.000243,...,0.000243,0.000243,0.000243,0.000243,0.213006,0.000243,0.056443,0.000243,0.000243,0.000243
4,xXXIACCJ2io,1263.360,0.309381,0.000269,0.158191,0.067313,0.112894,0.078968,0.000269,0.000269,...,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2959,eWZVAazaDco,2711.030,0.186967,0.000251,0.000251,0.124950,0.060717,0.000251,0.000251,0.000251,...,0.000251,0.000251,0.000251,0.021842,0.000251,0.000251,0.000251,0.000251,0.004754,0.000251
2960,eWZVAazaDco,3012.300,0.074200,0.159856,0.000267,0.000267,0.000267,0.000267,0.000267,0.019113,...,0.164314,0.276957,0.000267,0.000267,0.000267,0.000267,0.000267,0.015913,0.000267,0.000267
2961,eWZVAazaDco,3313.559,0.000301,0.000301,0.000301,0.000301,0.000301,0.000301,0.018603,0.000301,...,0.000301,0.000301,0.000301,0.000301,0.767786,0.000301,0.000301,0.000301,0.000301,0.000301
2962,eWZVAazaDco,3614.590,0.000273,0.192212,0.000273,0.508347,0.000273,0.000273,0.000273,0.000273,...,0.000273,0.000273,0.000273,0.000273,0.294796,0.000273,0.000273,0.000273,0.000273,0.000273


In [ ]:
# what are the words that make up each topic?

In [51]:

tmp_df = pd.DataFrame(lda.components_, columns=vectorizer.get_feature_names()).iloc[:,40:60]
tmp_df

/Users/Jay/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,abut,academia,academic,academy,acai,accelerate,acceleration,accelerometer,accent,acceptable,acceptance,accepted,accepting,access,accessibility,accessible,accessory,accident,acclamation,acclimate
0,0.05,0.050000,0.050000,0.05,0.05,0.050000,0.050000,0.050000,1.404892,1.266594,0.05,0.05,0.05,41.657685,0.05,0.050000,2.05,11.142818,0.050000,0.05
1,0.05,1.483498,0.050000,0.05,0.05,0.050000,0.050000,0.050000,0.050000,4.505425,0.05,0.05,0.05,7.238983,4.05,8.452929,0.05,0.050000,0.050000,0.05
2,0.05,0.050000,0.050000,0.05,0.05,0.050000,1.969407,0.050000,1.504084,0.050000,0.05,2.05,0.05,0.050000,0.05,0.050000,0.05,2.338619,0.050000,2.05
3,0.05,0.050000,1.358675,1.05,0.05,0.050000,0.050000,0.050000,0.050000,0.050000,0.05,0.05,0.05,34.801591,0.05,2.879413,0.05,0.050000,0.050000,0.05
4,1.05,0.050000,0.050000,0.05,0.05,0.050000,0.050000,0.050000,0.050000,0.050000,0.05,0.05,0.05,1.569244,0.05,0.050000,0.05,5.763489,4.271124,0.05
5,0.05,0.050000,0.050000,0.05,0.05,0.050000,5.184007,0.050000,0.050000,0.050000,0.05,0.05,0.05,0.050000,0.05,0.050000,0.05,0.050000,0.050000,0.05
6,0.05,0.050000,0.050000,0.05,0.05,0.050000,2.001627,1.759844,0.050000,0.050000,0.05,0.05,0.05,19.870067,0.05,2.749494,0.05,0.050000,0.050000,0.05
7,0.05,0.050000,0.050000,0.05,0.05,0.050000,0.050000,0.050000,0.050000,0.050000,0.05,0.05,0.05,0.050000,0.05,0.050000,0.05,0.050000,0.050000,0.05
8,0.05,0.050000,0.050000,0.05,0.05,1.272337,37.314753,4.130265,3.241024,1.404629,0.05,0.05,2.05,0.102153,0.05,6.841409,0.05,5.108672,0.050000,0.05
9,0.05,0.050000,0.050000,0.05,0.05,6.634516,57.789303,0.050000,0.050000,0.050000,0.05,0.05,1.05,4.142433,0.05,1.436607,0.05,0.050000,0.050000,0.05


In [ ]:
tmp_df.iloc[0,:]